# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os
import csv

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = "../WeatherPy/city_results.cvs"

city_results_df = pd.read_csv(csv_path, index_col="City Name")

city_results_df.head()


,City Number,City Country,Latitude,Longitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,City Datetime
City Name,,,,,,,,,
Tauá,3386568,BR,-5.9667,-40.3000,79.90,60,71,5.59,1613255606
Ushuaia,3833367,AR,-54.8000,-68.3000,44.60,81,75,0.56,1613255218
Hithadhoo,1282256,MV,-0.6000,73.0833,82.26,76,36,11.10,1613255269
Cherskiy,2126199,RU,68.7500,161.3000,-10.59,82,31,10.96,1613255270
Almaznyy,582447,RU,48.0448,40.0450,17.60,57,20,20.13,1613255610


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

#latitude and longitude in locations
locations = city_results_df[["Latitude","Longitude"]]

#humidity as weight
rating = city_results_df["Humidity"]

#Plot Heatmap
new_york_coordinates = (40.75, -74.00)
fig = gmaps.figure(center=new_york_coordinates, zoom_level=2)

#create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating,
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

#add layer
fig.add_layer(heat_layer)

#display
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
vacation_destinations = city_results_df
vacation_destinations


,City Number,City Country,Latitude,Longitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,City Datetime
City Name,,,,,,,,,
Tauá,3386568,BR,-5.9667,-40.3000,79.90,60,71,5.59,1613255606
Ushuaia,3833367,AR,-54.8000,-68.3000,44.60,81,75,0.56,1613255218
Hithadhoo,1282256,MV,-0.6000,73.0833,82.26,76,36,11.10,1613255269
Cherskiy,2126199,RU,68.7500,161.3000,-10.59,82,31,10.96,1613255270
Almaznyy,582447,RU,48.0448,40.0450,17.60,57,20,20.13,1613255610
...,...,...,...,...,...,...,...,...,...
Carutapera,3402648,BR,-1.1950,-46.0200,77.86,91,100,6.96,1613256236
Sangar,2017215,RU,63.9242,127.4739,-14.67,89,100,5.82,1613256238
Johannesburg,993800,ZA,-26.2023,28.0436,68.00,90,100,6.22,1613256239


In [5]:
drop_cities = vacation_destinations[vacation_destinations["Max Temperature"] > 80].index
vacation_destinations.drop(drop_cities, inplace=True)
vacation_destinations

,City Number,City Country,Latitude,Longitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,City Datetime
City Name,,,,,,,,,
Tauá,3386568,BR,-5.9667,-40.3000,79.90,60,71,5.59,1613255606
Ushuaia,3833367,AR,-54.8000,-68.3000,44.60,81,75,0.56,1613255218
Cherskiy,2126199,RU,68.7500,161.3000,-10.59,82,31,10.96,1613255270
Almaznyy,582447,RU,48.0448,40.0450,17.60,57,20,20.13,1613255610
Hermanus,3366880,ZA,-34.4187,19.2345,68.00,89,0,6.29,1613255225
...,...,...,...,...,...,...,...,...,...
Carutapera,3402648,BR,-1.1950,-46.0200,77.86,91,100,6.96,1613256236
Sangar,2017215,RU,63.9242,127.4739,-14.67,89,100,5.82,1613256238
Johannesburg,993800,ZA,-26.2023,28.0436,68.00,90,100,6.22,1613256239


In [6]:
drop_cities = vacation_destinations[vacation_destinations["Max Temperature"] < 70].index
vacation_destinations.drop(drop_cities, inplace=True)
vacation_destinations

,City Number,City Country,Latitude,Longitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,City Datetime
City Name,,,,,,,,,
Tauá,3386568,BR,-5.9667,-40.3000,79.90,60,71,5.59,1613255606
Port Elizabeth,964420,ZA,-33.9180,25.5701,73.40,94,75,8.05,1613255249
São José da Coroa Grande,3388456,BR,-8.8978,-35.1478,78.82,81,39,10.27,1613255628
Hualmay,3939761,PE,-11.0964,-77.6139,77.04,62,84,11.65,1613255336
Bac Lieu,1591474,VN,9.2850,105.7244,72.18,83,75,9.13,1613255631
...,...,...,...,...,...,...,...,...,...
Ankazoabo,1072879,MG,-22.2833,44.5167,70.38,84,100,9.22,1613256227
São João da Barra,3448903,BR,-21.6403,-41.0511,78.80,78,0,13.80,1613255933
Alice Springs,2077895,AU,-23.7000,133.8833,71.60,78,3,5.75,1613256125


In [7]:
drop_cities = vacation_destinations[vacation_destinations["Wind Speed"] > 10].index
vacation_destinations.drop(drop_cities, inplace=True)
vacation_destinations

,City Number,City Country,Latitude,Longitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,City Datetime
City Name,,,,,,,,,
Tauá,3386568,BR,-5.9667,-40.3000,79.90,60,71,5.59,1613255606
Port Elizabeth,964420,ZA,-33.9180,25.5701,73.40,94,75,8.05,1613255249
Bac Lieu,1591474,VN,9.2850,105.7244,72.18,83,75,9.13,1613255631
Ahipara,2194098,NZ,-35.1667,173.1667,72.41,75,99,8.77,1613255637
Dingle,1714733,PH,10.9995,122.6711,74.97,91,100,6.73,1613255209
...,...,...,...,...,...,...,...,...,...
Gualaquiza,3657661,EC,-3.4000,-78.5500,72.84,79,56,1.97,1613256225
Ankazoabo,1072879,MG,-22.2833,44.5167,70.38,84,100,9.22,1613256227
Alice Springs,2077895,AU,-23.7000,133.8833,71.60,78,3,5.75,1613256125


In [8]:
drop_cities = vacation_destinations[vacation_destinations["Cloud Coverage"] > 20].index
vacation_destinations.drop(drop_cities, inplace=True)
vacation_destinations

,City Number,City Country,Latitude,Longitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,City Datetime
City Name,,,,,,,,,
Singkang,1732871,MY,2.1006,102.7994,75.20,88,20,4.61,1613255665
Grand Gaube,934479,MU,-20.0064,57.6608,75.99,80,0,5.99,1613255253
Carnarvon,2074865,AU,-24.8667,113.6333,75.20,100,9,9.22,1613255227
Bela Vista,3470177,BR,-22.1089,-56.5211,75.00,83,7,0.87,1613255421
Wukari,2318921,NG,7.8500,9.7833,78.48,16,2,1.74,1613255747
Santa Isabel,3450188,BR,-23.3156,-46.2214,70.00,94,20,4.61,1613255813
Meulaboh,1214488,ID,4.1363,96.1285,76.71,84,5,4.36,1613255716
Vao,2137773,NC,-22.6667,167.4833,78.75,77,0,6.13,1613255314
Chui,3443061,UY,-33.6971,-53.4616,70.81,91,16,7.29,1613255581


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
vacation_destinations['Hotel Name'] = ""
vacation_destinations = vacation_destinations.head(7)
vacation_destinations

,City Number,City Country,Latitude,Longitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,City Datetime,Hotel Name
City Name,,,,,,,,,,
Singkang,1732871,MY,2.1006,102.7994,75.20,88,20,4.61,1613255665,
Grand Gaube,934479,MU,-20.0064,57.6608,75.99,80,0,5.99,1613255253,
Carnarvon,2074865,AU,-24.8667,113.6333,75.20,100,9,9.22,1613255227,
Bela Vista,3470177,BR,-22.1089,-56.5211,75.00,83,7,0.87,1613255421,
Wukari,2318921,NG,7.8500,9.7833,78.48,16,2,1.74,1613255747,
Santa Isabel,3450188,BR,-23.3156,-46.2214,70.00,94,20,4.61,1613255813,
Meulaboh,1214488,ID,4.1363,96.1285,76.71,84,5,4.36,1613255716,


In [10]:
#Set parameters to search for hotels with 5000 meters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
              "rankby" : "distance",
              "type" : "lodging",
              "key" : g_key,
            }

In [11]:
for index, row in vacation_destinations.iterrows():
    #get lattitude and logitude from df
    lat = row["Latitude"]
    long = row["Longitude"]
    latlong = f'{lat},{long}'
    
    params["location"] = latlong
    
    response = requests.get(base_url, params=params).json()    
    
    print (response)
    results = response['results']
    
    #print(f'lat = {lat} long = {long}')
    try:
        vacation_destinations.loc[index,'Hotel Name'] = results[0]['name']
        
    except:
        print('--------')
        
vacation_destinations.reset_index()

{'html_attributions': [], 'next_page_token': 'ATtYBwLfxhyTTrq3gj1aiLPovOitKXQb-B3XF3WjOAI4lO4RcZ9a7vGVdQ9lUWR5q7_J6Iwja-GppMuC2XUW292xhBI0fLy_JQsptGsQuR6PUJli4aloS3tRo8d1T_lNulvoRI4d3rBhY5_xoIg543HNImBNmN3D6NQBXNpO4tHoDdxZ-y17c9lzP_HVCRoM8Qnbe2K51qyPyNZO82OJPxZQFoZOzmcxImbNJ65PqEmCSxDXwXJvao2rx6QuGBSKX6fYSiWvmCb0DZlwia9VO1vhXTj_R9aRNjwJCM3ZQpqMzxI9nCRE5Jk1x2ueafcdD6d-Rk6BI7cRyrnK6MHVNfGOoqhWWY4TscJDcL2RTm_wVa_z0LJnGM03q5yYd4CCcy1i7b0TzWmnYvyYvmobdbKYpaIWmkdyGDJUEtuCVPaRDbSpjHcERKLPmRQ0V0Q', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 2.1223061, 'lng': 102.7664561}, 'viewport': {'northeast': {'lat': 2.123630180291502, 'lng': 102.7677840802915}, 'southwest': {'lat': 2.120932219708498, 'lng': 102.7650861197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Kampung Seri Makmor Pagoh', 'photos': [{'height': 3000, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/106355823270188992443">

C:\Users\rache\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


{'html_attributions': [], 'next_page_token': 'ATtYBwKB4QFOiu2ADF2Grwe7WGI6FDBToatGMXiMWvRMqPp_0L2o6rZgp3EpSJagRKMuMjF0c7uyJo9T5Od91Mo3rDzpJZmr6Gy0sdhmUxAGTptBzYKBXXy_S2VCIsn9hvBcXJCFoWdkcZExFesndp0o7LN9FlA6Tkk8V444h3lQQmTkVtTdrwkQKBwFA8djdvdfLDnngyOugqlK0g_4u0KtG7JgrIupgNnTwKPtzb3IzidQBR4bRjIc8XRPXTv7hcNcAQPKXcdq50IN6UhVHjOjfGDpKcglV9NrgT7DnK1GkCUllZrkUkC-OLLZfFnlvTTOLGbUEzD9dyN0Jbh9skVn7aHq2ZfNt2zOs-3_p6OBaJCwGjPOMWHawY3ZBpAy8n-98TL7G4FbXTtge0ZhnPxoegDqahMBVfD6PjBUrMhF3R8vG1IbqNnwyxmpQpA', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -20.0075143, 'lng': 57.6612178}, 'viewport': {'northeast': {'lat': -20.0061275697085, 'lng': 57.66249668029151}, 'southwest': {'lat': -20.0088255302915, 'lng': 57.6597987197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Talipot Lodge', 'photos': [{'height': 4032, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/117894418423043897193">Gregory Soob

{'html_attributions': [], 'next_page_token': 'ATtYBwIuRmxNL4Eq8WyiYwu0rtFCFt6z9nJBZV2x5cPVt4_e2VKkbAtAqqk2WUEko9SrKkc7y5YlwVEdpq3cBP7-LynA7aiBcBp5jmR5NndlyBDmpFhuHZ6J31DIM8MY-M3Xy324Be4SWFHts5qgPmxYBGgAT9Jt3pZ6lfg2QTKk9LVapd_4kpc4eNBFRsed4lvsJ9BMWh5JrI99OTeVPp2M2IsCvXwlctghYIlqx5Gdwe8zYgDBCZETvMlb1TOFmYOFWE26CCiYC4RqKO-OgFfJNkkOMVQ6Ox6MQZv8OnWumU0-I7RvlLSMj2A_32tiaKASVsv9n6jDRf18FK7uLl4yMhxbNmZmZxCWVUQMZwazSr8HF0pa7rsDIE3OOtz8DXEtaVz8BWLqOfaYLPWKgsEHY2CdsEJ5JQqyDZ9pSYWXWl-ZWA1xsqvXCT2_Il0', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -24.8800197, 'lng': 113.6525195}, 'viewport': {'northeast': {'lat': -24.87862161970849, 'lng': 113.6539914802915}, 'southwest': {'lat': -24.8813195802915, 'lng': 113.6512935197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Tilly Gia home sweet home', 'place_id': 'ChIJQ0UrptvG4ysRijT0kEZJ-MA', 'plus_code': {'compound_code': '4M93+X2 Brockman WA, Australia', 'global_c

{'html_attributions': [], 'next_page_token': 'ATtYBwKY8Bpv8tv8EhjZ48XTVHCA9s9MvgsvngZg727bhc9hUhtyFXAULCnLTNRoce8SSqm1Z_rLsGh9gnWB35dhGReKZQVfktSw5tkTbXvKzGjZvs-qwsQ4-Lls5uePegrScT5Q8TJfjPSOJluyNQhHxOEkaASw3CXkv7oNFr32P-pclJqmoTH_K1-o9KogcpXMgZR-Lgk6j1vaARToWJcc8Jv6MmidRe1DjshQQroeD_zHuXaz08YWYKt1JUnYIq2Yi8LAPt9R_ZZ3BpM-6iraH7BdSGILJLkwew6KXdyP2bHx1EWaIHuHaQvfW6IvjVl5mBm0qN6S5oHTkexWqwePC90zstfzfAFeHeTLsbJhcuGdysJORLsUm96c7Nhr-RsaAZue3IImaY3JAayngV7aXNXPNh1kgM1wINdLO1n5uqa5C49BsSAeqYLqD-s', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -22.1089877, 'lng': -56.5224567}, 'viewport': {'northeast': {'lat': -22.1077595697085, 'lng': -56.5211312697085}, 'southwest': {'lat': -22.1104575302915, 'lng': -56.52382923029149}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Pousada Panorama', 'photos': [{'height': 4160, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/117922221482003903339">Oscar 

{'html_attributions': [], 'next_page_token': 'ATtYBwLrccAHsLghAfOxXp3Nh9epqaczPoAufI6MkeOMyUB5ei62VYW_7JaHh98eQ_vONObpJoacomE3385r3gKjzvpXC1F8dMD1xq7XpULcoEBBDzt6oUN76T-Q-gg7tPo6bzKApiADQPyodMGN7aMZM1Ngix8gv6fHZosix8v5PWY_JkzaWqsaV6pExn3f-jUkw5cjlQkxPrSGyxMFSEnjhSMwX0Lf48Em31a4ehWjon4clqSSj_sp8-g9vQg79PG_D6Grw6HgS0qODvqNcVGiqOaI7JOKt_m2MJXRFBRTTSAXLN_fyrzoe4FEgjLS9fIwOk0tySd860S8ZVVSVZ49Nq1boynj9DO2JTSBtpVGwc3St58MqChkOwYv3J4f9MuFWByhppYGFBM3iwWATcqQSH5i4Gyeme2Itu38yrYTJzpMTW9bVuzIyQ952PQ', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 7.8508598, 'lng': 9.7799873}, 'viewport': {'northeast': {'lat': 7.852319080291503, 'lng': 9.781312880291503}, 'southwest': {'lat': 7.849621119708498, 'lng': 9.778614919708499}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Karofi Lodge', 'photos': [{'height': 4160, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/104708360203625432744">Robert Bisong</

{'html_attributions': [], 'next_page_token': 'ATtYBwLj2LMIOblx9zjrTOCtROlqa9uCPCm-gxYwSPWzceHsz7yiNtNbmHzLe5I493iEn4c_Lme4YFqfk5M424X8rNVJxHHsRQ1LogkL4rF15NaVuEgPzamxzQ_kCWXdKGLabEnUzpmoP6YwCGj2QCxNvy9sF-bCRmiXrUCqsKuRN4WnescE03KIIJ2-le_MWY5PfJJTKHk3QfC9iZgTFbFIkYmOjDuBzfL9iGtJM5DRJTKEnVkP2uw3qZprJOdf3OAojsS9kRhyHEPR-IvLh6MR4WASJlz00LMvMnduEG67v83e3EGcS13NtgnpaPXuTjD1Uamzx_SrvkTGQ22PZMZqdN6Xky_gVAbIyoVWoUpe8ldF3efzCNGyvaOHNoRupfBu2txu5zw3WlZJz017FNdEhLFTRT8a0920RP6QEXwpnSYrEeLxzQXazMt0xCw', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -23.3167239, 'lng': -46.2210338}, 'viewport': {'northeast': {'lat': -23.3153770197085, 'lng': -46.2196865697085}, 'southwest': {'lat': -23.31807498029151, 'lng': -46.2223845302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Casa Pena Branca', 'place_id': 'ChIJK3cgAMWBzpQRzgeQqQcD62c', 'plus_code': {'compound_code': 'MQMH+8H Jardim Japao, Santa Isabel - SP, Brazil', '

{'html_attributions': [], 'next_page_token': 'ATtYBwL7YwRt5aiL0b11ix1PE-UWU20CqLW2CMy1zJ5pka9yEK3quJdmR5Fv60iKokPrQQggoG2Tb7lknmsqp8OdirBx_dh-SYBwyyPYlo5aGRBKCCp85mKjydU6QQcAFad_e0dofQANAD75cuVZ01SO5CvQCPi4Kz1EOziwUCwbZQ2ZDox72-qyyaIfs3qQ43E606m5EnIXhjbaKoy04Lt6PK4MPH14Xz3Bz2sQ1UzF729gnLYb2HmYAMZpNtMri5HYdMElzcARHl6XGcX4RaTO42cc2nWWr7KSwNnVFxAVkxoPsM2OoPN21euDRlSj7lwQeOqvHVZRNPxBY2zbqeUHpz0ftIZTKQygMSExhaqQPT_vgHUQhwy6tyYKXZFCSXET2swAEqPDuSACAopoUCyVrrFUUqfGjxOz17h1VruAOEBvkK0RK4DCq22Ezx8', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 4.1372898, 'lng': 96.12685499999999}, 'viewport': {'northeast': {'lat': 4.138667180291502, 'lng': 96.1281935302915}, 'southwest': {'lat': 4.135969219708498, 'lng': 96.12549556970849}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Wisma seulanga', 'opening_hours': {'open_now': True}, 'place_id': 'ChIJCyjtpp3DPjARblM7LuTWphw', 'plus_code': {'compound_code': '44PG+WP Kampun

,City Name,City Number,City Country,Latitude,Longitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,City Datetime,Hotel Name
0,Singkang,1732871,MY,2.1006,102.7994,75.20,88,20,4.61,1613255665,Kampung Seri Makmor Pagoh
1,Grand Gaube,934479,MU,-20.0064,57.6608,75.99,80,0,5.99,1613255253,Talipot Lodge
2,Carnarvon,2074865,AU,-24.8667,113.6333,75.20,100,9,9.22,1613255227,Tilly Gia home sweet home
3,Bela Vista,3470177,BR,-22.1089,-56.5211,75.00,83,7,0.87,1613255421,Pousada Panorama
4,Wukari,2318921,NG,7.8500,9.7833,78.48,16,2,1.74,1613255747,Karofi Lodge
5,Santa Isabel,3450188,BR,-23.3156,-46.2214,70.00,94,20,4.61,1613255813,Casa Pena Branca
6,Meulaboh,1214488,ID,4.1363,96.1285,76.71,84,5,4.36,1613255716,Wisma seulanga


In [12]:
hotel_df = vacation_destinations
hotel_df.reset_index(inplace=True)
hotel_df.rename(columns={'Latitude':'Lat',
                         'Longitude' : 'Lng',
                         'City Name' : 'City',
                         'City Country' : "Country"
                        }, inplace=True)
hotel_df = hotel_df.head()

C:\Users\rache\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
##hotel_info = [info_box_template.format(**row) for index, row in vacation_destinations.iterrows()]
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
#locations = vacation_destinations[["Latitude", "Longitude"]]
locations.head()

,Lat,Lng
0,2.1006,102.7994
1,-20.0064,57.6608
2,-24.8667,113.6333
3,-22.1089,-56.5211
4,7.8500,9.7833


In [14]:
hotel = hotel_df["Hotel Name"].tolist()
city = hotel_df["City"].tolist()
country = hotel_df["Country"].tolist()

In [15]:
# Display figure
fig

marker_locations = hotel_df[['Lat', 'Lng']]

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(marker_locations, info_box_content = [f"Hotel: {x}  &  City: {y}  &  Country: {z}" for x,y,z in zip(hotel,city,country)])

fig.add_layer(markers)


# Display figure
fig


Figure(layout=FigureLayout(height='420px'))